In [14]:
from google.colab import drive
drive.mount('/content/gdrive')

! cp "/content/gdrive/My Drive/InfoFaceRecognize/dataset/mydataset.zip" .
! cp "/content/gdrive/My Drive/InfoFaceRecognize/weights/last_model_mnv2.h5" .
! unzip mydataset.zip

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
Archive:  mydataset.zip
replace train/43.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


### Import Libs ###

In [15]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import f1_score, accuracy_score
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt
from PIL import ImageFile
import pandas as pd
import numpy as np
import random

### Set Params ###

In [16]:
ImageFile.LOAD_TRUNCATED_IMAGES = True
IMG_PATH = ""
IMAGE_WIDTH = 224
IMAGE_HEIGHT = 224
from PIL import ImageFile
IMAGE_SIZE = (IMAGE_WIDTH, IMAGE_HEIGHT)
BATCH_SIZE = 128
EPOCHS = 150

### Load and transform data ###

In [17]:
test_df = pd.read_csv("fairface_label_new_val.csv")
test_df.head()

,file,age,gender,race,service_test
0,val/1.jpg,3-9,Male,East Asian,False
1,val/2.jpg,50-59,Female,East Asian,True
2,val/4.jpg,20-29,Female,Latino_Hispanic,True
3,val/5.jpg,20-29,Male,Southeast Asian,False
4,val/6.jpg,30-39,Male,Latino_Hispanic,False


### Load model ###

In [18]:
face_model = load_model('last_model_mnv2.h5')

### Create predict generator ###

In [19]:
test_datagen = ImageDataGenerator(
    rescale=1./255
)

In [20]:
test_pred = test_datagen.flow_from_dataframe(
    test_df, 
    IMG_PATH,
    x_col='file',
    y_col=None,
    class_mode=None,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    shuffle=False,
)

Found 7468 validated image filenames.


In [21]:
nb_samples = test_df.shape[0]

### Predict ###

In [22]:
race_label = {x: y for x, y in enumerate(sorted(test_df.race.unique()))}
gender_label = {x: y for x, y in enumerate(sorted(test_df.gender.unique()))}
age_label = {x: y for x, y in enumerate(sorted(test_df.age.unique()))}

In [23]:
age_label

{0: '0-2',
 1: '10-19',
 2: '20-29',
 3: '3-9',
 4: '30-39',
 5: '40-49',
 6: '50-59',
 7: '60-69',
 8: 'more than 70'}

In [24]:
predictions = face_model.predict(test_pred, steps=np.ceil(nb_samples/BATCH_SIZE))

### Evaluation ###

In [25]:
test_df['race_pred'] = np.argmax(predictions[0], axis=-1)
test_df['race_pred'] = test_df['race_pred'].replace(race_label)
f1_race = f1_score(test_df.race, test_df.race_pred, average='weighted')
acc_race = accuracy_score(test_df.race, test_df.race_pred)

print(f"""
    F1-Score: {np.round(f1_race*100, 2)}%
    Accuracy: {np.round(acc_race*100, 2)}%
""")


    F1-Score: 68.77%
    Accuracy: 68.88%



In [26]:
test_df['gender_pred'] = np.argmax(predictions[1], axis=-1)
test_df['gender_pred'] = test_df['gender_pred'].replace(gender_label)
f1_gender = f1_score(test_df.gender, test_df.gender_pred, average='weighted')
acc_gender = accuracy_score(test_df.gender, test_df.gender_pred)

print(f"""
    F1-Score: {np.round(f1_gender*100, 2)}%
    Accuracy: {np.round(acc_gender*100, 2)}%
""")


    F1-Score: 92.35%
    Accuracy: 92.35%



In [32]:
test_df['age_pred'] = np.argmax(predictions[2], axis=-1)
test_df['age_pred'] = test_df['age_pred'].replace(age_label)
f1_age = f1_score(test_df.age, test_df.age_pred, average='weighted')
acc_age = accuracy_score(test_df.age, test_df.age_pred)

print(f"""
    F1-Score: {np.round(f1_age*100, 2)}%
    Accuracy: {np.round(acc_age*100, 2)}%
""")


    F1-Score: 57.11%
    Accuracy: 57.82%

